## Packages

In [2]:
import shutil
import json
import csv
import os
import re
from natsort import natsorted
from tqdm import tqdm

## Label data analysis

In [3]:
%pwd

'C:\\Users\\Suin\\Workspace\\motion-to-emotion'

In [4]:
%cd data

C:\Users\Suin\Workspace\motion-to-emotion\data


In [5]:
with open('Training/L_SIT/20201030_dog-sit-000273.mp4.json', encoding='utf-8') as f:
    label_sample = json.load(f)

In [6]:
print(label_sample.keys())

name, meta, anno = label_sample.values()
print(f'''
video name: {name}
seq #     : {meta['seq']}
action    : {meta['action']}
emotion   : {meta['inspect']['emotion']}
height    : {meta['height']}
width     : {meta['width']}
frames    : {len(anno)}
          ''')

dict_keys(['file_video', 'metadata', 'annotations'])

video name: dog-sit-000273.mp4
seq #     : 273
action    : 앉기
emotion   : 행복/즐거움
height    : 1280
width     : 720
frames    : 86
          


In [7]:
anno[0]['bounding_box']

{'x': 6, 'y': 311, 'width': 592, 'height': 766}

## Data preprocessing
    - Train: 39537
    - Val  : 2474
    - Test : 2475

In [9]:
action_list = [ 'BODYLOWER', 'BODYSCRATCH', 'BODYSHAKE', 'FEETUP', 'FOOTUP', 'HEADING',
                'LYING', 'MOUNTING', 'SIT', 'TAILING', 'TAILLOW', 'TURN', 'WALKRUN' ]
emo_list = [ '공격성', '공포', '불안/슬픔', '편안/안정', '행복/즐거움', '화남/불쾌' ]

In [ ]:
##########################################################################
# label.csv
#-------------------------------------------------------------------------
# filename : '{seq}_{frame}.jpg'
# seq      : sequence ID number; representing same video source
# frame_idx: frame ID number; 0, 1, 2, ...
# action   : action class number
# emotion  : emotion class number
# bbox     : bounding box; (x1, y1, x2, y2); starting/ending point
# keypoints: keypoints; [(x, y, v), ... ], v for visibility
##########################################################################

In [30]:
for i in range(2):
    source = ['Training', 'Validation'][i]
    target = ['train', 'val'][i]
    
    label_log = open(os.path.join(target, 'label.csv'), 'w', newline='')
    lr = csv.writer(label_log)
    lr.writerow(['file_name', 'seq', 'frame_idx', 'action', 'emotion', 'bbox', 'keypoints'])
    
    error_log = open(os.path.join(target, 'error.csv'), 'w', newline='')
    er = csv.writer(error_log)
    
    # for each Action folder
    for i, act in enumerate(action_list):
        path_label = os.path.join(source, f'L_{act}')
        path_origin= os.path.join(source, f'O_{act}')

        # for each video
        for video in tqdm(os.listdir(path_origin), desc=f'{act:10}'):
            if video == '.ipynb_checkpoints':
                continue

            # extract informations from label file
            try:
                with open(f'{path_label}/{video}.json', 'r', encoding='utf-8') as f:
                    label = json.load(f)
                    _, meta, anno = label.values()
                    seq = int(meta['seq'])
                    emotion = emo_list.index(meta['inspect']['emotion'])
            except Exception as e:
                er.writerow([e])
                continue

            # for each frame
            for j, frame_name in enumerate(natsorted(os.listdir(f'{path_origin}/{video}'))):
                # find annotation index
                temp = re.split('[_.]', frame_name)
                N_fr = int(temp[1])
                N_ts = int(temp[3])
                img_name = f'{seq}_{N_fr}.jpg'

                # get annotation data
                keys = dict()
                bbox = dict()
                for frame in anno:
                    if frame['frame_number'] == N_fr and frame['timestamp'] == N_ts:
                        keys = frame['keypoints']
                        bbox = frame['bounding_box']
                        break

                # bounding box
                x1, y1, w, h = bbox.values()
                x2 = x1 + w
                y2 = y1 + h

                # keypoints
                keylist = []
                for key in keys.values():
                    keylist.append([key['x'], key['y'], 1] if type(key) == dict else [0, 0, 0])

                # copy images to new directory and rename
                shutil.move(f'{path_origin}/{video}/{frame_name}', f'{target}/{img_name}')

                # label.csv logging
                lr.writerow([img_name, seq, j, i, emotion, [x1, y1, x2, y2], keylist])
    label_log.close()
    error_log.close()

FOOTUP    :  72%|███████████████████████████████████████████████▍                  | 2982/4154 [00:44<00:17, 67.09it/s]